In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
import torch

## Convert Json to Pandas Dataframe

In [2]:
train = pd.read_json("data/train-v1.1.json")

In [3]:
train.shape

(442, 2)

In [4]:
train.head(3)

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1


In [5]:
train.iloc[1,0]['paragraphs'][0]

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
   'id': '56be85543aeaaa14008c9063',
   'question': 'When did Beyonce start becoming popular?'},
  {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
   'id': '56be85543aeaaa14008c9065',
   'ques

In [6]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(100):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [7]:
df.shape

(22618, 4)

In [8]:
df.to_csv("data/train.csv", index = None)

## Create dictionary of sentence embeddings for faster computation

In [9]:
paras = list(df["context"].drop_duplicates().reset_index(drop= True))

In [10]:
len(paras)

5068

In [11]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [12]:
len(sentences)

20075

In [13]:
infersent = torch.load('InferSent/encoder/infersent.allnli.pickle', map_location=lambda storage, loc: storage)
infersent.set_glove_path("InferSent/dataset/GloVe/glove.840B.300d.txt")

In [14]:
infersent.build_vocab(sentences, tokenize=True)

Found 37124(/42258) words with glove vectors
Vocab size : 37124


In [ ]:
dict_embeddings = {}
for i in range(len(sentences)):   
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

In [17]:
with open('data/dict_embeddings.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)